In [1]:
#Results were produced in stints. This is the number of the last stint.
run = 11

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from scipy import stats
from scipy import optimize
import joblib

#folder for saving results
filepath = ".../Resultate_final/VarAnn/IS/VarAnn_GlueVaR_IS_sim_saved/"

In [3]:
#parameters as in section 4.3 of 'Assessing Asset-Liability risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
#and section 5.1 of 'A Least-Squares Monte Carlo Approach to the Estimation of Enterprise Risk' (Ha, Bauer 2019)
#parameters for q
q_0 = 4.605
m = 0.05
sigma_S = 0.18
#parameters for r
r_0 = 0.025
zeta = 0.25
gamma = 0.02
sigma_r = 0.01
lambd = 0.02
gamma_bar = gamma - (lambd*sigma_r)/zeta
#parameters for mu_(55+t)
mu_55 = 0.01
kappa = 0.07
sigma_mu = 0.0012
#parameters for brownian motion
rho_12 = -0.3
rho_13 = 0.06
rho_23 = -0.04
cov_mat = np.array([[1,rho_12,rho_13],[rho_12,1,rho_23],[rho_13,rho_23,1]])
#horizon parameters
tau = 1
T = 15
b = 10.792
#GlueVaR parameters
alpha_Glue = 0.95
beta_Glue = 0.995
omega_Glue = np.array([1/3,1/3])

In [4]:
#Sizes for training, validation, test, and set size for Monte Carlo estimation of the risk measures
M_1 = 1500000
M_2 = 500000
M_3 = 500000
#ignore N or N_2 in the following. Was kept just in case, but not used.
N_2 = 1
M_MC = 500000
#size of the set of data points used to calculate an IS density
M_IS = 750000
#quantile for which the IS density will be computed
alpha_IS = 0.975

In [5]:
#Definition of these functions analogously to the appendix of 'A Least-Squares Monte Carlo Approach to the Estimation of Enterprise Risk' (Ha, Bauer 2019)
def B_r(t,T):
    return ((1-np.exp(-zeta*(T-t)))/zeta)
def B_mu(t,T):
    return ((np.exp(kappa*(T-t))-1)/kappa)
def A(t,T):
    tmp1 = gamma_bar*(B_r(t,T)-(T-t))
    tmp2 = (sigma_r/zeta)**2 * ((T-t) - 2*B_r(t,T) + (1-np.exp(-2*zeta*(T-t)))/(2*zeta))
    tmp3 = (sigma_mu/kappa)**2 * ((T-t) - 2*B_mu(t,T) + (np.exp(2*kappa*(T-t))-1)/(2*kappa))
    tmp4 =  2*rho_23*sigma_r*sigma_mu/(zeta*kappa) * (B_mu(t,T) - (T-t) + B_r(t,T) - (1-np.exp(-(zeta-kappa)*(T-t)))/(zeta-kappa))
    return np.exp(tmp1+0.5*(tmp2+tmp3+tmp4))
#Definition of this function analogously to section 4.3 of 'Assessing Asset-Liability risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
def F(t,k,r_t,mu_xt):
    return (A(t,t+k)*np.exp(-B_r(t,t+k)*r_t - B_mu(t,t+k)*mu_xt))

#parameters of the normal distribution of X_tau according to the appendix of 'A Least-Squares Monte Carlo Approach to the Estimation of Enterprise Risk' (Ha, Bauer 2019)
mu_q_tau = q_0 + (m-0.5*(sigma_S**2))*tau
mu_r_tau = r_0*np.exp(-zeta*tau) + gamma*(1-np.exp(-zeta*tau))
mu_mu_55_tau = mu_55*np.exp(kappa*tau)
mean_tau = np.array([mu_q_tau, mu_r_tau, mu_mu_55_tau])

cov_q_r_tau = rho_12*sigma_S*sigma_r*B_r(0,tau)
cov_q_mu_tau = rho_13*sigma_S*sigma_mu*B_mu(0,tau)
cov_r_mu_tau = rho_23*sigma_r*sigma_mu* ((1-np.exp(-(zeta-kappa)*tau))/(zeta-kappa))
var_q_tau = (sigma_S**2) * tau
var_r_tau = (sigma_r**2) * ((1-np.exp(-2*zeta*tau))/(2*zeta))
var_mu_tau = (sigma_mu**2) * ((np.exp(2*kappa*tau)-1)/(2*kappa))
cov_mat_tau = np.array([[var_q_tau, cov_q_r_tau, cov_q_mu_tau], [cov_q_r_tau, var_r_tau, cov_r_mu_tau], [cov_q_mu_tau, cov_r_mu_tau, var_mu_tau]])
C_tau = np.linalg.cholesky(cov_mat_tau)

#variance/covariance parameters of the conditional normal distribution of X_T according to the appendix of 'A Least-Squares Monte Carlo Approach to the Estimation of Enterprise Risk' (Ha, Bauer 2019)
var_q_T_cond = (sigma_S**2) *(T-tau) + ((sigma_r/zeta)**2) * (T-tau - 2*(1-np.exp(-zeta*(T-tau)))/zeta + (1-np.exp(-2*zeta*(T-tau)))/(2*zeta)) + (2*rho_12*sigma_S*sigma_r/zeta) * (T-tau- (1-np.exp(-zeta*(T-tau)))/zeta)
cov_q_r_T_cond = rho_12*sigma_S*sigma_r*((1-np.exp(-zeta*(T-tau)))/zeta) + ((sigma_r**2)/zeta) * ((1-2*np.exp(-zeta*(T-tau))+np.exp(-2*zeta*(T-tau)))/(2*zeta))
cov_q_mu_T_cond = rho_13*sigma_S*sigma_mu *((np.exp(kappa*(T-tau))-1)/kappa) + (rho_23*sigma_r*sigma_mu/zeta) * ((np.exp(kappa*(T-tau))-1)/kappa - (1-np.exp(-(zeta-kappa)*(T-tau)))/(zeta-kappa))
var_r_T_cond = (sigma_r**2) * ((1-np.exp(-2*zeta*(T-tau)))/(2*zeta))
cov_r_mu_T_cond = rho_23*sigma_r*sigma_mu*((1-np.exp(-(zeta-kappa)*(T-tau)))/(zeta-kappa))
var_mu_T_cond = (sigma_mu**2) * ((np.exp(2*kappa*(T-tau))-1)/(2*kappa))
cov_mat_T_cond = np.array([[var_q_T_cond, cov_q_r_T_cond, cov_q_mu_T_cond], [cov_q_r_T_cond, var_r_T_cond, cov_r_mu_T_cond], [cov_q_mu_T_cond, cov_r_mu_T_cond, var_mu_T_cond]])

#function for generating simulated risk factors X_tau and corresponding payments Y from multivariate standard normal random variables
def data_gen(Z,V):
    #simulation of X_tau
    X_tau = np.transpose(np.matmul(C_tau,np.transpose(Z))) + np.tile(mean_tau, (len(Z),1))

    #simulation of X_T given X_tau
    mu_q_T_cond = X_tau[:,0] + B_r(tau,T)*X_tau[:,1] + (gamma_bar - (sigma_r/zeta)**2)*(T-tau - (1-np.exp(-zeta*(T-tau)))/zeta) + 0.5*((sigma_r/zeta)**2) * ((1-np.exp(-zeta*(T-tau)))/zeta - (np.exp(-zeta*(T-tau))-np.exp(-2*zeta*(T-tau)))/zeta) - ((rho_23*sigma_r*sigma_mu)/kappa) * ( (np.exp(kappa*(T-tau))-1)/(kappa*(zeta-kappa)) - (np.exp(kappa*(T-tau))-np.exp(-(zeta-kappa)*(T-tau)))/(zeta*(zeta-kappa)) - (1/zeta) * (T-tau - (1-np.exp(-zeta*(T-tau)))/zeta)) -0.5*(sigma_S**2) * (T-tau) - (rho_12*sigma_S*sigma_r/zeta) * (T-tau - (1-np.exp(-zeta*(T-tau)))/zeta) - (rho_13*sigma_S*sigma_mu/kappa) * ((np.exp(kappa*(T-tau))-1)/kappa -T+tau)
    mu_r_T_cond = np.exp(-zeta*(T-tau))*X_tau[:,1] + (gamma_bar-(sigma_r/zeta)**2)*(1-np.exp(-zeta*(T-tau))) + 0.5*((sigma_r/zeta)**2) *(1-np.exp(-2*zeta*(T-tau))) - (rho_23*sigma_r*sigma_mu/kappa) * ((1-np.exp(-(zeta-kappa)*(T-tau)))/(zeta-kappa) - (1-np.exp(-zeta*(T-tau)))/zeta)
    mu_mu_T_cond = np.exp(kappa*(T-tau))*X_tau[:,2] - (rho_23*sigma_r*sigma_mu/zeta) * ((np.exp(kappa*(T-tau))-1)/kappa - (1-np.exp(-(zeta-kappa)*(T-tau)))/(zeta-kappa)) - ((sigma_mu**2)/kappa) * ((np.exp(2*kappa*(T-tau))-1)/(2*kappa) - (np.exp(kappa*(T-tau))-1)/kappa)
    mean_T_cond = np.array([mu_q_T_cond, mu_r_T_cond, mu_mu_T_cond])

    X_T = V + np.transpose(mean_T_cond)
    
    #calculation of Y from X_T and X_tau
    Y = F(t=tau, k=T-tau, r_t=X_tau[:,1], mu_xt=X_tau[:,2]) * np.maximum(np.exp(X_T[:,0]), b*np.sum([F(t=T, k=i, r_t=X_T[:,1], mu_xt=X_T[:,2]) for i in range(1,51)], axis=0))
    return X_tau, Y

#the function DT(Z,\theta)
def data_trans_IS(Z,IS):
    res = np.empty((len(Z),3))
    for j in range(3):
        res[:,j] = Z[:,j]*np.sqrt(IS[3+j]) + IS[j]
    return res

#The density function of Z
def f(y):
    return stats.multivariate_normal.pdf(y, mean=np.full(3,0), cov=np.identity(3))

#The density function of Z_\theta (note that x is interpreted as theta, needed for the least-squares solver to work properly)
def f_theta(y, x):
    return stats.multivariate_normal.pdf(y, mean=x[0:3], cov=np.diag(x[3:6]))

#This function describes the approximation of the expression inside the sum of m_2(theta)
def g_q_alpha_hat_reweighted(x,L,q_alpha_hat):
    return [(np.sqrt(f(y=L[i,0:3])/f_theta(x=x, y=L[i,0:3])) if L[i,-1]>q_alpha_hat else 0) for i in range(len(L))]

#bounds for the IS density parameters (for the parameters corresponding to the mean no bounds are necessary, the standard deviation parameters however needs to be non-negative)
bnds_lower = np.array([-np.inf, -np.inf, -np.inf, 0,0,0])
bnds_upper = np.full(6,np.inf)
bnds = (bnds_lower, bnds_upper)

#function for calculating GlueVaR in an IS setting
def GlueVaR_IS(omega, L, alpha, beta, w):
    j_beta = 0
    w_sum_tmp = 0
    while(w_sum_tmp <= (1-beta)):
        w_sum_tmp += w[j_beta]
        j_beta += 1
        
    j_alpha = j_beta
    while(w_sum_tmp <= (1-alpha)):
        w_sum_tmp += w[j_alpha]
        j_alpha += 1
        
    ES_beta = 1/(1-beta) * np.sum(w[0:j_beta-1]*L[0:j_beta-1]) + ( 1 - (1 / (1-beta)) * np.sum(w[0:j_beta-1]) )*L[j_beta]
    ES_alpha = 1/(1-alpha) * np.sum(w[0:j_alpha-1]*L[0:j_alpha-1]) + ( 1 - (1 / (1-alpha)) * np.sum(w[0:j_alpha-1]) )*L[j_alpha]
    VaR_alpha = L[j_alpha]

    return omega[0]*ES_beta + omega[1]*ES_alpha + (1-omega[0]-omega[1])*VaR_alpha

In [6]:
for j in range(100):
    #Generating realisations of multivariate standard normal random variables, V correlated
    Z_IS = np.random.multivariate_normal(mean=np.full(3,0), cov=np.identity(3), size=M_IS)
    V_IS = np.random.multivariate_normal(mean=np.full(3,0), cov=cov_mat_T_cond, size=M_IS)

    #Calculate the risk factor X_tau and the corresponding simulated payoffs Y
    X_tau_IS, Y_IS = data_gen(Z=Z_IS, V=V_IS)

    #define and compile neural network model, setup as in section 4.3 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
    bi_IS = np.log( np.sum(Y_IS)/len(Y_IS))
    model_IS = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(3,)),
        tf.keras.layers.Dense(4, activation='tanh'),
        tf.keras.layers.Dense(4, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi_IS))])
    model_IS.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model_IS.fit(x=X_tau_IS, y=Y_IS, epochs=40, batch_size=10000, verbose=0)

    #Calculate realisations of L_hat from the training data set using the trained neural network
    L_hat_IS = np.column_stack((Z_IS, model_IS.predict(X_tau_IS)[:,0]))
    L_hat_IS_sort = L_hat_IS[L_hat_IS[:,-1].argsort()[::-1]]

    #Calculating the corresponding estimator for Value-at-Risk in order to approximate g
    q_alpha_IS_hat = L_hat_IS_sort[int(M_IS*(1-alpha_IS)-1), -1]
    print('q_alpha_IS_hat_NN:',q_alpha_IS_hat)

    #Calculating the (hopefully) approximately optimal \theta^*_{NN} by minimising m_2 using the approximated g
    IS_NN = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.concatenate((np.full(3,0),np.full(3,1))), args=(L_hat_IS, q_alpha_IS_hat), bounds=bnds).x

    #define and train a random forest according to the optimal parameters from tuning
    rfr_IS = RandomForestRegressor(n_estimators=160, criterion='squared_error', max_features=2, min_samples_leaf=1300, bootstrap=True, verbose=0, n_jobs=-1)
    rfr_IS.fit(X=X_tau_IS, y=Y_IS)

    #Calculate realisations of L_hat from the training data set using the trained random forest
    L_hat_IS = np.column_stack((Z_IS, rfr_IS.predict(X_tau_IS)))
    L_hat_IS_sort = L_hat_IS[L_hat_IS[:,-1].argsort()[::-1]]

    #Calculating the corresponding estimator for Value-at-Risk in order to approximate g
    q_alpha_IS_hat = L_hat_IS_sort[int(M_IS*(1-alpha_IS)-1), -1]
    print('q_alpha_IS_hat_RF:',q_alpha_IS_hat)

    #Calculating the (hopefully) approximately optimal \theta^*_{RF} by minimising m_2 using the approximated g
    IS_RF = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.concatenate((np.full(3,0),np.full(3,1))), args=(L_hat_IS, q_alpha_IS_hat), bounds=bnds).x

    #print IS density parameters for checking
    print('IS_NN:',IS_NN)
    print('IS_RF:',IS_RF)
    
    #Generating simulations for multivariate standard normal random variables (uncorrelated for Z and correlated for V) for training set, validation set, test set, set for Monte Carlo estimation of risk measures
    Z_train = np.random.multivariate_normal(mean=np.full(3,0), cov=np.identity(3), size=M_1)
    V_train = np.random.multivariate_normal(mean=np.full(3,0), cov=cov_mat_T_cond, size=M_1)
    Z_val = np.random.multivariate_normal(mean=np.full(3,0), cov=np.identity(3), size=M_2)
    V_val = np.random.multivariate_normal(mean=np.full(3,0), cov=cov_mat_T_cond, size=M_2)
    Z_test = np.random.multivariate_normal(mean=np.full(3,0), cov=np.identity(3), size=M_3)
    V_test = np.random.multivariate_normal(mean=np.full(3,0), cov=cov_mat_T_cond, size=M_3)
    Z_MC = np.random.multivariate_normal(mean=np.full(3,0), cov=np.identity(3), size=M_MC)
    V_MC = np.random.multivariate_normal(mean=np.full(3,0), cov=cov_mat_T_cond, size=M_MC)

    #calculate DT(Z,\theta^*_{NN})
    Z_train_NN = data_trans_IS(Z_train,IS_NN)
    Z_val_NN = data_trans_IS(Z_val,IS_NN)
    Z_test_NN = data_trans_IS(Z_test,IS_NN)
    Z_MC_NN = data_trans_IS(Z_MC,IS_NN)
    #calculating the risk factors under the IS distribution and corresponding realised payoffs
    X_tau_train_NN, Y_train_NN = data_gen(Z=Z_train_NN, V=V_train)
    X_tau_val_NN, Y_val_NN = data_gen(Z=Z_val_NN, V=V_val)
    X_tau_test_NN, Y_test_NN = data_gen(Z=Z_test_NN, V=V_test)
    X_tau_MC_NN, Y_MC_NN = data_gen(Z=Z_MC_NN, V=V_MC)

    #calculate DT(Z,\theta^*_{RF})
    Z_train_RF = data_trans_IS(Z_train,IS_RF)
    Z_val_RF = data_trans_IS(Z_val,IS_RF)
    Z_test_RF = data_trans_IS(Z_test,IS_RF)
    Z_MC_RF = data_trans_IS(Z_MC,IS_RF)
    #calculating the risk factors under the IS distribution and corresponding realised payoffs
    X_tau_train_RF, Y_train_RF = data_gen(Z=Z_train_RF, V=V_train)
    X_tau_val_RF, Y_val_RF = data_gen(Z=Z_val_RF, V=V_val)
    X_tau_test_RF, Y_test_RF = data_gen(Z=Z_test_RF, V=V_test)
    X_tau_MC_RF, Y_MC_RF = data_gen(Z=Z_MC_RF, V=V_MC)
    
    #calculating parameters for the sets B_1 and B_2
    q_70 = stats.norm.ppf(0.7, loc=mu_q_tau, scale=var_q_tau)
    q_30 = stats.norm.ppf(0.3, loc=mu_q_tau, scale=var_q_tau)
    r_70 = stats.norm.ppf(0.7, loc=mu_r_tau, scale=var_r_tau)
    r_30 = stats.norm.ppf(0.3, loc=mu_r_tau, scale=var_r_tau)

    #calculate the indices of the set B_1 and B_2 for the test set created with the IS density calculated by the neural network
    B_1_NN = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (X_tau_test_NN[:,0] > q_70, X_tau_test_NN[:,1] < r_30)) )
    B_2_NN = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (X_tau_test_NN[:,0] < q_30, X_tau_test_NN[:,1] > r_70)) )

    #calculate the indices of the set B_1 and B_2 for the test set created with the IS density calculated by the random forest
    B_1_RF = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (X_tau_test_RF[:,0] > q_70, X_tau_test_RF[:,1] < r_30)) )
    B_2_RF = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (X_tau_test_RF[:,0] < q_30, X_tau_test_RF[:,1] > r_70)) )
    
    #define and compile neural network model, setup as in section 4.3 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
    bi = np.log( np.sum(Y_train_NN)/len(Y_train_NN))
    model = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(3,)),
        tf.keras.layers.Dense(4, activation='tanh'),
        tf.keras.layers.Dense(4, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi))])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #training the neural network
    hist = model.fit(x=X_tau_train_NN, y=Y_train_NN, epochs=40, batch_size=10000, validation_data=(X_tau_val_NN,Y_val_NN), verbose=0)
    
    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 for the neural network
    Y_pred_NN = model.predict(X_tau_test_NN)[:,0]
    mse_train_NN = hist.history['mse'][-1]
    mse_val_NN = hist.history['val_mse'][-1]
    mc_tmp = Y_pred_NN - Y_test_NN
    metric_a_NN = np.sum(mc_tmp)/len(Y_test_NN)
    metric_b_NN = np.sum((mc_tmp)*Y_pred_NN)/len(Y_test_NN)
    metric_c_B_1_NN = np.sum(mc_tmp[B_1_NN])/len(Y_test_NN)
    metric_c_B_2_NN = np.sum(mc_tmp[B_2_NN])/len(Y_test_NN)

    #computation of expected payoffs depending on the risk factor X_tau according to the models, i.e. computation of L_hat_i's
    L_hat_NN = model.predict(X_tau_MC_NN)[:,0]
    L_hat_c_NN = np.column_stack((Z_MC_NN, L_hat_NN))
    
    #calculation of the IS estimator for GlueVaR
    L_hat_c_sort_NN = L_hat_c_NN[L_hat_c_NN[:,-1].argsort()[::-1]]
    w = f(L_hat_c_sort_NN[:,0:3])/(M_MC*f_theta(x=IS_NN, y=L_hat_c_sort_NN[:,0:3]))

    GlueVaR_hat_NN = GlueVaR_IS(omega=omega_Glue, L=L_hat_c_sort_NN[:,-1], alpha=alpha_Glue, beta=beta_Glue, w=w)
    print('GlueVaR_hat_NN:',GlueVaR_hat_NN)
    
    
    #perform a grid search in order to find the (approximately) best hyperparameter min_samples_leaf
    #values that will be checked
    max_features_list = [2]
    min_samples_leaf_list = [2250,2500,2750]
    opt_param = np.full(2,0)
    opt_score = np.inf

    for max_features in max_features_list:
        for min_samples_leaf in min_samples_leaf_list:
            rfr_tuning = RandomForestRegressor(n_estimators=160, max_features=max_features, min_samples_leaf=min_samples_leaf, bootstrap=True, criterion='squared_error', verbose=0, n_jobs=-1)
            rfr_tuning.fit(X=X_tau_train_RF, y=Y_train_RF)
            score = mean_squared_error(y_true=Y_val_RF, y_pred=rfr_tuning.predict(X_tau_val_RF))
            if score < opt_score:
                opt_param_RF = np.array([max_features,min_samples_leaf])
                opt_score = score
    
    #definition and training of random forest regressor
    rfr = RandomForestRegressor(n_estimators=400, criterion='squared_error', max_features=int(opt_param_RF[0]), min_samples_leaf=int(opt_param_RF[1]), bootstrap=True, verbose=0, warm_start=True, n_jobs=-1)
    rfr.fit(X=X_tau_train_RF, y=Y_train_RF)
    
    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 and training/valdiation MSE for the random forest
    mse_train_RF = mean_squared_error(y_true=Y_train_RF, y_pred=rfr.predict(X_tau_train_RF))
    mse_val_RF = mean_squared_error(y_true=Y_val_RF, y_pred=rfr.predict(X_tau_val_RF))
    Y_pred_RF = rfr.predict(X_tau_test_RF)
    mc_tmp = Y_pred_RF - Y_test_RF
    metric_a_RF = np.sum(mc_tmp)/len(Y_test_RF)
    metric_b_RF = np.sum((mc_tmp)*Y_pred_RF)/len(Y_test_RF)
    metric_c_B_1_RF = np.sum(mc_tmp[B_1_RF])/len(Y_test_RF)
    metric_c_B_2_RF = np.sum(mc_tmp[B_2_RF])/len(Y_test_RF)

    #computation of the expected payoff depending on the risk factor X_tau according to the models, i.e. computation of L_hat_i's
    L_hat_RF = rfr.predict(X_tau_MC_RF)
    L_hat_c_RF = np.column_stack((Z_MC_RF, L_hat_RF))
    
    #calculation of the IS estimator for GlueVaR
    L_hat_c_sort_RF = L_hat_c_RF[L_hat_c_RF[:,-1].argsort()[::-1]]
    w = f(L_hat_c_sort_RF[:,0:3])/(M_MC*f_theta(x=IS_RF, y=L_hat_c_sort_RF[:,0:3]))

    GlueVaR_hat_RF = GlueVaR_IS(omega=omega_Glue, L=L_hat_c_sort_RF[:,-1], alpha=alpha_Glue, beta=beta_Glue, w=w)
    print('GlueVaR_hat_RF:',GlueVaR_hat_RF)
    
    #save results for further evaluation
    output = np.array([[mse_train_NN,mse_val_NN,metric_a_NN,metric_b_NN,metric_c_B_1_NN,metric_c_B_2_NN,IS_NN[0],IS_NN[1],IS_NN[2],IS_NN[3],IS_NN[4],IS_NN[5],GlueVaR_hat_NN],
                      [mse_train_RF,mse_val_RF,metric_a_RF,metric_b_RF,metric_c_B_1_RF,metric_c_B_2_RF,IS_RF[0],IS_RF[1],IS_RF[2],IS_RF[3],IS_RF[4],IS_RF[5],GlueVaR_hat_RF]])

    joblib.dump(output,filepath+'output_'+str(run)+'_'+str(j)+'.joblib')
    #prints just for checking while the notebook is running
    print(j)

2021-11-06 14:19:45.922827: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-06 14:19:54.220195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:16:00.0, compute capability: 7.0
2021-11-06 14:19:54.344716: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30988 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3a:00.0, compute capability: 7.0
2021-11-06 14:19:54.346262: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:

q_alpha_IS_hat_NN: 129.76986694335938
q_alpha_IS_hat_RF: 129.19310908430504
IS_NN: [ 1.98574138 -0.14414745 -0.93999126  0.39113847  0.9794789   1.10746994]
IS_RF: [ 1.89814568 -0.15282591 -0.9265567   0.45136758  1.00446446  1.24918008]
GlueVaR_hat_NN: 133.3975610364167
GlueVaR_hat_RF: 134.93183007731025
0
q_alpha_IS_hat_NN: 128.48057556152344
q_alpha_IS_hat_RF: 129.45409256818772
IS_NN: [ 1.91643443 -0.1826985  -1.06361598  0.42924211  0.89690288  1.06219126]
IS_RF: [ 2.055648   -0.09424235 -0.77423162  0.30643986  0.95705446  1.03333483]
GlueVaR_hat_NN: 134.39774302907455
GlueVaR_hat_RF: 134.58893066204763
1
q_alpha_IS_hat_NN: 129.32962036132812
q_alpha_IS_hat_RF: 128.96330047229546
IS_NN: [ 1.90998738 -0.23890163 -1.04667255  0.41240773  0.93050306  1.06916966]
IS_RF: [ 1.90905244 -0.14097516 -0.92525491  0.42075976  1.0614694   1.33765117]
GlueVaR_hat_NN: 133.48155894847872
GlueVaR_hat_RF: 134.44858742300084
2
q_alpha_IS_hat_NN: 126.6164779663086
q_alpha_IS_hat_RF: 129.31628505902

KeyboardInterrupt: 